In [5]:
import torch
import torch.nn as nn

# 2D input
input_shape = (4, 5)  # 10 rows, 20 columns
x = torch.randn(1, *input_shape)  # 1 is the batch size

print(x)

# Linear layer
linear = nn.Linear(in_features=5, out_features=30)
output = linear(x)

print(output.shape)  # Output: torch.Size([1, 10, 30])
pr

tensor([[[ 0.2760,  1.2130,  1.0729,  0.4244,  0.3087],
         [ 0.6272,  1.2952,  0.1131, -1.3732, -0.4800],
         [ 0.2979, -1.1214,  0.0164,  1.4246, -0.5989],
         [-1.2109,  0.2366,  0.7696, -0.8035, -0.8255]]])
torch.Size([1, 4, 30])


In [6]:
output

tensor([[[ 0.1905, -0.2360, -0.2053, -0.2610, -0.3912,  0.3399, -0.6973,
           0.1033,  0.9741,  0.5628,  0.0389, -0.1824, -0.3992, -0.7927,
          -0.1711, -0.5420, -0.4703,  0.0291, -0.3251,  0.1695,  0.8731,
          -1.0046,  0.1572, -0.0994,  0.5664, -0.3688, -0.0816,  0.6668,
           0.8375, -0.4091],
         [ 0.4102,  0.6772, -0.1949,  0.6795,  0.1661,  0.6836, -0.3199,
          -0.8748,  0.6855, -0.0701,  0.4740, -0.4025,  0.1223, -0.1498,
          -0.6818, -0.2329,  0.4690, -0.1503,  0.6310,  0.8566,  0.0665,
          -0.5087, -0.5925, -0.8389,  0.1517, -0.2474,  0.2444,  0.8240,
           0.2003, -0.0240],
         [ 0.5474, -0.5512, -0.0831, -0.8365,  0.2020, -0.4148, -0.4491,
           0.3547, -0.0040, -0.1837,  0.2674,  1.0462, -0.4612, -0.5069,
           0.3118, -0.7054, -1.3046,  0.2161, -0.3874, -0.1488, -0.1115,
          -0.9281,  1.4109,  0.0278,  1.3912,  0.8302, -0.8739, -0.0183,
          -0.0905,  0.6142],
         [ 1.3419, -0.3146, -0.6512, 